In [1]:
import pymskt as mskt
from pymskt.mesh import BoneMesh, CartilageMesh
import os
import SimpleITK as sitk

In [2]:
location_seg = '../../data/right_knee_example.nrrd'

In [3]:
# initiate the bone mesh object
femur = BoneMesh(path_seg_image=location_seg,  # path to the segmentation iamge being used
                 label_idx=5,                  # what is the label of this bone
                 list_cartilage_labels=[1])    # a list of labels for cartialge associated w/ this bone

# Create the bone mesh
femur.create_mesh(
    smooth_image_var=1.0       # This is the variance of the gaussian filter applied to binary image b4 meshing
)

# Resample the bone surface to have a specified number of nodes. 
femur.resample_surface(
    clusters=20000             # This is the number of nodes/vertices on the surface. It might vary slightly
)

# Calcualte cartialge thickness for the cartialge meshes associated with the bone
femur.calc_cartilage_thickness(
    image_smooth_var_cart=0.5   # variance for the gaussian filter applied to the binary cart meshes b4 smoothing. 
)


# read in the segmentation image & break it up into femoral cartilage sub-regions. 
seg_image = sitk.ReadImage(location_seg)
seg_image = mskt.image.cartilage_processing.get_knee_segmentation_with_femur_subregions(seg_image)

# update bone segmentation (with fem cartilage subregions) 
femur.seg_image = seg_image
femur.list_cartilage_labels=[11, 12, 13, 14, 15]

# assign the cartilage regions to the surface
femur.assign_cartilage_regions()

# IF you want to save the mesh you can uncomment the below line
# femur.save_mesh('path/to_save/mesh.vtk') # I choose VTK becuase it preserves the scalars - STL does not. 




/Users/gattia/opt/miniconda3/envs/imaging/lib/python3.8/site-packages/pyvista/core/dataset.py:1401: PyvistaDeprecationWarning: Use of `point_arrays` is deprecated. Use `point_data` instead.
  warnings.warn(
/Users/gattia/opt/miniconda3/envs/imaging/lib/python3.8/site-packages/pyvista/core/dataset.py:1541: PyvistaDeprecationWarning: Use of `cell_arrays` is deprecated. Use `cell_data` instead.
  warnings.warn(


In [4]:
from itkwidgets import view

# Note that we have to get the mesh from the femur object `femur.mesh` to input it into itkwidgets. 
# In this notebook, you should be able to swap between scalars (thickness & labels)
# thickness is the cartilage thickness in mm, labels is the cartilage label of the associated thickness value. 

# In this notebook, the labels are all cartilage sub-regions. (trochlea, medial weight-bearing femur, 
# lateral weight-bearing femur, medial posterior femur, and lateral posterior femur). The notebook 
# Create_Femur_Mesh_Assign_Cartilage_Thickness_and_Regions_Feb.3.2022.ipynb includes just one label for 
# all femoral cartilage
view(geometries=[femur.mesh])

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…